In [38]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [39]:
file_path = (
    "./jsonl/gpt-4o-third-run-results-batch_7IQQpDsgKAHHxisiqlSobQKq_output.jsonl"
)
gt_data = pd.read_json(f"./images_third_prompt//configurations.json")

In [40]:
# Dictionary to map custom_id to message content
custom_id_to_content = {}

# Open the file and read line by line
with open(file_path, "r") as file:
    for line in file:
        json_obj = json.loads(line)
        # Extract custom_id and message content
        custom_id = json_obj.get("custom_id")
        message_content = json_obj["response"]["body"]["choices"][0]["message"][
            "content"
        ]
        # Map custom_id to message content
        if custom_id:
            custom_id_to_content[custom_id.replace("uid__", "")] = message_content

# Print the dictionary to verify
print(len(custom_id_to_content))

150


In [41]:
gt_data = (
    gt_data.T
)  # use indexx as a new column in the dataframe and called it image_path
gt_data = gt_data.reset_index()
gt_data = gt_data.rename(columns={"index": "image_path"})

# ad a new column to gt_data with content of  custom_id_to_content (match key with image_path)
gt_data["message"] = gt_data["image_path"].map(custom_id_to_content)

In [42]:
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness,message
0,image_0_thickness_2.png,3,1,"[red, blue]",2,The blue and red line plots cross each other t...
1,image_0_thickness_3.png,3,1,"[red, blue]",3,The blue and red line plots cross each other o...
2,image_0_thickness_4.png,3,1,"[red, blue]",4,The blue and red line plots cross each other o...
3,image_1_thickness_2.png,3,2,"[red, blue]",2,The blue and red line plots cross each other t...
4,image_1_thickness_3.png,3,2,"[red, blue]",3,The blue and red line plots cross each other t...
...,...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3,The blue and red line plots cross each other t...
146,image_48_thickness_4.png,3,2,"[red, blue]",4,The blue and red line plots cross each other t...
147,image_49_thickness_2.png,3,0,"[red, blue]",2,The blue and red line plots cross each other t...
148,image_49_thickness_3.png,3,0,"[red, blue]",3,The blue and red line plots cross each other t...


In [43]:
# remove nan
gt_data = gt_data.dropna()

In [44]:
# import re


# # Function to extract numbers from text, including zero represented as text
# def extract_number(text):
#     # Check for phrases that imply zero intersections
#     if (
#         "no intersections" in text
#         or "zero line intersections" in text
#         or "do not intersect" in text
#         or "intersect zero times" in text
#     ):
#         return 0
#     # First try to find digits
#     match = re.search(r"\b\d+\b", text)
#     if match:
#         return int(match.group())
#     # If no digits, look for text numbers (one, two, three, etc.)
#     text_to_num = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5}
#     for word, num in text_to_num.items():
#         if word in text:
#             return num
#     # Check for phrases that imply multiple intersections
#     if (
#         "intersect twice" in text
#         or "intersect in two places" in text
#         or "intersect two times" in text
#     ):
#         return 2
#     if (
#         "intersect once" in text
#         or "intersect at one point" in text
#         or "intersect one time" in text
#         or "intersect only once" in text
#         or "intersect at a single point" in text
#     ):
#         return 1
#     return None


# # Apply the function to the 'message' column
# gt_data["extracted_number"] = gt_data["message"].apply(extract_number)
# gt_data[["image_path", "message", "extracted_number"]]

# # Count none or na in gt_data extracted_number
# na_count = gt_data["extracted_number"].isna().sum()
# print(f"Number of NA/None values: {na_count}")

# # Show their value in the message (same row)
# na_messages = gt_data[gt_data["extracted_number"].isna()]["message"]
# print("Messages with NA/None extracted numbers:")
# print(na_messages)

In [45]:
import re


def extract_marked_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Regular expression to find numbers (digits or words)
    number_words = "zero|one|two|three|four|five|six|seven|eight|nine|ten"
    pattern = rf"\b({number_words}|\d+)\b"

    # Mapping from words to digits
    number_map = {
        "zero": "0",
        "one": "1",
        "two": "2",
        "three": "3",
        "four": "4",
        "five": "5",
        "six": "6",
        "seven": "7",
        "eight": "8",
        "nine": "9",
        "ten": "10",
    }

    # Search for the first occurrence of a number
    match = re.search(pattern, text)
    if match:
        # Map the word to its corresponding digit if it's a word
        return number_map.get(match.group(0), match.group(0))
    else:
        # Check for implied single intersection
        if "intersect once" in text or "intersect only once" in text:
            return "1"
        # Check for no intersections
        elif (
            "do not intersect" in text
            or "not intersect" in text
            or "no intersection" in text
        ):
            return "0"
        # Check for multiple intersections
        elif "intersect twice" in text:
            return "2"
        elif "intersect three times" in text:
            return "3"
        # Default case if no clear intersection information is found
        elif "once" in text:
            return "1"

    if "do not cross" in text:
        return "0"
    elif "twice" in text:
        return "2"
    elif "three times" in text:
        return "3"
    elif "single crossing" in text:
        return "1"
    elif "never cross" in text:
        return "0"
    else:
        return "number_not_found" + text


gt_data["extracted_number"] = gt_data["message"].apply(extract_marked_text)

In [46]:
gt_data["extracted_number"].value_counts()

extracted_number
2    84
1    50
3     9
0     7
Name: count, dtype: int64

In [47]:
# count none or na in gt_data extracted_number
gt_data["extracted_number"].isna().sum()
# show their value in the message (same row)
gt_data[gt_data["extracted_number"].isna()]["message"]

Series([], Name: message, dtype: object)

In [48]:
# Cast num_intersections to integer
gt_data["num_intersections"] = gt_data["num_intersections"].astype(int)

# Cast extracted_number to integer (it should already be integer from the extraction function, but just to ensure consistency)
gt_data["extracted_number"] = gt_data["extracted_number"].astype(int)

# Compare num_intersections and extracted_number to check if they are the same
gt_data["is_correct"] = gt_data["num_intersections"] == gt_data["extracted_number"]

# Display the DataFrame with the new column
gt_data[["image_path", "num_intersections", "extracted_number", "is_correct"]]

,image_path,num_intersections,extracted_number,is_correct
0,image_0_thickness_2.png,1,2,False
1,image_0_thickness_3.png,1,1,True
2,image_0_thickness_4.png,1,1,True
3,image_1_thickness_2.png,2,2,True
4,image_1_thickness_3.png,2,2,True
...,...,...,...,...
145,image_48_thickness_3.png,2,2,True
146,image_48_thickness_4.png,2,2,True
147,image_49_thickness_2.png,0,2,False
148,image_49_thickness_3.png,0,2,False


In [49]:
gt_data["is_correct"].value_counts()

is_correct
False    78
True     72
Name: count, dtype: int64

In [50]:
100 * np.mean(gt_data["is_correct"])

48.0

In [51]:
gt_data

,image_path,num_points,num_intersections,line_colors,line_thickness,message,extracted_number,is_correct
0,image_0_thickness_2.png,3,1,"[red, blue]",2,The blue and red line plots cross each other t...,2,False
1,image_0_thickness_3.png,3,1,"[red, blue]",3,The blue and red line plots cross each other o...,1,True
2,image_0_thickness_4.png,3,1,"[red, blue]",4,The blue and red line plots cross each other o...,1,True
3,image_1_thickness_2.png,3,2,"[red, blue]",2,The blue and red line plots cross each other t...,2,True
4,image_1_thickness_3.png,3,2,"[red, blue]",3,The blue and red line plots cross each other t...,2,True
...,...,...,...,...,...,...,...,...
145,image_48_thickness_3.png,3,2,"[red, blue]",3,The blue and red line plots cross each other t...,2,True
146,image_48_thickness_4.png,3,2,"[red, blue]",4,The blue and red line plots cross each other t...,2,True
147,image_49_thickness_2.png,3,0,"[red, blue]",2,The blue and red line plots cross each other t...,2,False
148,image_49_thickness_3.png,3,0,"[red, blue]",3,The blue and red line plots cross each other t...,2,False


In [52]:
# Assuming 'gt_data' DataFrame has columns 'line_thickness', 'num_intersections', and 'extracted_number'

# Convert line_thickness to an appropriate numeric type if necessary
gt_data["line_thickness"] = gt_data["line_thickness"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = gt_data.groupby("line_thickness").apply(
    lambda df: (df["num_intersections"] == df["extracted_number"]).mean()
)

# Print the accuracy by line thickness
# print(accuracy_by_thickness
#
# #)
# multiply by 100 and round 2 decimal
accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_39737/1027132002.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = gt_data.groupby("line_thickness").apply(


In [53]:
print(accuracy_by_thickness.to_latex())

\begin{tabular}{lr}
\toprule
 & 0 \\
line_thickness &  \\
\midrule
2 & 52.000000 \\
3 & 48.000000 \\
4 & 44.000000 \\
\bottomrule
\end{tabular}



# Export

In [54]:
gt_data["Model"] = ["gpt-4o"] * len(gt_data)
# rename message to model_output_raw
gt_data.rename(columns={"message": "model_output_raw"}, inplace=True)
gt_data.to_pickle("./data/gpt-4o-3.pkl")